In [1]:
import pymysql

mydb = pymysql.connect(
    host= 'localhost',
    user= 'root',
    password= 'QWERTY#123#'
)

# mycursor = mydb.cursor()
# mycursor.execute('create database budget_tracker')
# mycursor.execute('show databases')
# for x in mycursor:
#     print(x)

In [2]:
mydb = pymysql.connect(
    host= 'localhost',
    user= 'root',
    password= 'QWERTY#123#',
    database= 'budget_tracker'
) 

mycursor = mydb.cursor()

# mycursor.execute('''    
#     create table budget(
#         ID int primary key not null auto_increment,
#         Date date not null,
#         Category varchar(200) not null,
#         Description text not null,
#         Amount varchar(200) not null,
#         Type varchar(100) not null,
#         PaymentMethod varchar(100) not null,
#         Status varchar(100) not null
#     )
# ''')
# mycursor.execute('''    
#     create table income(
#         ID int not null,
#         Amount varchar(200) not null,
#         Month varchar(50) not null,
#         Year varchar(10) not null
#     )
# ''')
# mycursor.execute('''    
#     create table income_tracker(
#         ID int not null,
#         Date date not null,
#         Amount varchar(200) not null,
#         Month varchar(50) not null,
#         Year varchar(10) not null
#     )
# ''')


mycursor.execute('show tables from budget_tracker')
for x in mycursor:
    print(x)




('budget',)
('income',)
('income_tracker',)


In [3]:
# mycursor.execute('''
#     alter table income add column Status text not null after Year
# ''')

# mycursor.execute('show columns from budget')
# for x in mycursor:
#     print(x)

mycursor.execute('show columns from income')
for x in mycursor:
    print(x)

('ID', 'int', 'NO', '', None, '')
('Amount', 'varchar(200)', 'NO', '', None, '')
('Month', 'varchar(50)', 'NO', '', None, '')
('Year', 'varchar(10)', 'NO', '', None, '')
('Status', 'text', 'NO', '', None, '')


In [4]:
query_update_status = 'UPDATE income SET Status = %s WHERE ID = %s'
mycursor.execute(query_update_status, ('Incomplete', 1))
mydb.commit()

In [6]:
from datetime import datetime
import locale

#____________________________________________________FUNCTIONS________________________________________#
def get_non_empty_input(prompt):
    while True:
        user_input = input(prompt)
        if user_input.strip(): 
            return user_input
        
def format_date_for_output(date_obj):
    day = date_obj.strftime("%d")
    month_name = date_obj.strftime("%b")
    year = date_obj.strftime("%Y")
    ordinal_indicator = get_ordinal_indicator(int(day))
    formatted_date = f"{day}{ordinal_indicator} {month_name}, {year}"
    return formatted_date

def get_ordinal_indicator(day):
    """Returns the appropriate ordinal indicator (st, nd, rd, th) for a given day number."""
    if 4 <= day % 100 <= 20 or day % 10 == 0:
        return "th"
    else:
        return {1: "st", 2: "nd", 3: "rd"}.get(day % 10, "th")
        
# def get_date_range():
#     # Format start and end dates
#     start_date_str = input("Enter start date (YYYY-MM-DD): ")
#     end_date_str = input("Enter end date (YYYY-MM-DD): ")
#     formatted_start_date = format_date_for_output(datetime.strptime(start_date_str, "%Y-%m-%d"))
#     formatted_end_date = format_date_for_output(datetime.strptime(end_date_str, "%Y-%m-%d"))
#     formatted_start_date = datetime.strptime(start_date_str, "%Y-%m-%d").strftime("%Y-%m-%d")
#     formatted_end_date = datetime.strptime(end_date_str, "%Y-%m-%d").strftime("%Y-%m-%d")
#     if formatted_end_date >= formatted_start_date:
#         return formatted_start_date, formatted_end_date
#     elif formatted_end_date < formatted_start_date:
#         print("End date must be equal to or after start date. Please try again.")
#     else:
#         print('Invalid date format. Please use YYYY-MM-DD.')
# formatted_start_date, formatted_end_date = get_date_range()
# print(f"Selected date range: {formatted_start_date} to {formatted_end_date}")

def format_with_commas(value, currency_symbol= '₦'):
    if isinstance(value, (int, float)):
        locale.setlocale(locale.LC_ALL, '')
        formatted_value = locale.format_string('%s%0.2f',(currency_symbol,value), grouping=True)
        return f'{currency_symbol}{formatted_value}'
    elif isinstance(value, str) and value.replace(',','').isdigit():
        numeric_value = float(value)
        locale.setlocale(locale.LC_ALL, '')
        formatted_value = locale.format_string('%s%0.2f',(currency_symbol,numeric_value), grouping=True)
        return formatted_value
    else:
        return value
    
def particular_month(Month):
    global balance
    Month = get_non_empty_input('Enter month of Expense: ')
    query = 'SELECT Amount FROM income where Month = %s'
    mycursor.execute(query, (Month))
    result = mycursor.fetchone()

balance = 0.0

                    #_________________________________ACCOUNTS___________________________________________________#
def main_balance():
    global balance
    Month = get_non_empty_input('Enter month of Expense: ')
    query = 'SELECT Amount FROM income where Month = %s'
    mycursor.execute(query, (Month))
    result = mycursor.fetchone()
    if result is None:
        print('Budget not found!')
    else:
        formatted_value = format_with_commas(result[0])
        print(f'Your main balance for {Month} is {format_with_commas(formatted_value)}')

def new_transfer():
    global balance 
    ID = 1
    Amount = float(input('Enter transfer amount: '))
    Month = get_non_empty_input('Enter Month of Transfer: ')
    Year = get_non_empty_input('Enter Year of Transfer: ')
    Status = get_non_empty_input('Enter Status of Budget(Completed/Incomplete): ')

    # Check if income already exists for the specified month and year
    query_check_income = 'SELECT * FROM income WHERE Month = %s AND Year = %s'
    mycursor.execute(query_check_income, (Month, Year))
    existing_income = mycursor.fetchone()

    if existing_income:
        print(f'Income already exists for {Month}, {Year}.')
        return

    # Retrieve the remaining amount from the previous month
    query_remaining_amount = 'SELECT Amount FROM income WHERE Month = %s AND Year = %s AND Status = %s'
    mycursor.execute(query_remaining_amount, (Month, Year, 'Incomplete'))
    remaining_amount = mycursor.fetchone()

    if remaining_amount:
        remaining_amount = remaining_amount[0]
    else:
        remaining_amount = 0.0

    # Add income to the balance and update the income table
    query_add_income = 'INSERT INTO income(ID, Amount, Month, Year, Status) VALUES (%s, %s, %s, %s, %s)'
    mycursor.execute(query_add_income, (ID, Amount + remaining_amount, Month, Year, Status))
    mydb.commit()

    # Update the budget table with the new income
    query_update_balance = 'UPDATE income SET Amount = Amount + %s WHERE Month = %s AND Year = %s and Status = %s'
    mycursor.execute(query_update_balance, (Amount + remaining_amount, Month, Year, Status))
    mydb.commit()

    balance += Amount + remaining_amount
    print(f'Transfer completed successfully...')
    #print(f'Your current balance is {format_with_commas(balance)}')



def transaction_history():
    mycursor.execute("SELECT ID, Amount, Month, Year, Status FROM income")
    records = mycursor.fetchall()
    for record in records:
        formatted_value = format_with_commas(record[1])
        formatted_record = list(record)
        formatted_record[1] = formatted_value
        print(formatted_record) 

def display_records(records):
    columns = [desc[0] for desc in mycursor.description]
    print(f"{' | '.join(columns)}")

    for record in records:
        formatted_values = [format_with_commas(val) for val in record]
        print(f"{' | '.join(map(str, formatted_values))}")

def add_income():
    """
    Adds income to a running total while maintaining its value across function calls.

    Args:
        initial_balance (float, optional): The starting balance. Defaults to 0.

    Returns:
        float: The updated balance after adding income.
    """
    # Date = input('Enter date(YYYY-MM-DD): ')
    global balance
    income =float(input('Enter deposit amount: '))
    balance += income
    print(f'Your current income is {format_with_commas(income)}...')
    print(f'Your current balance is {format_with_commas(balance)}')

def calculate_total_budget(Month):
    try:
        query = 'SELECT SUM(Amount) FROM budget WHERE Month = %s AND Status = %s'
        mycursor.execute(query, (Month, 'Pending'))
        result = mycursor.fetchone()

        return format_with_commas(result[0] if result[0] is not None else 0)
    except Exception as e:
        print(f"Error: {e}")
        return 0
    

def insight():
    global balance
    #start_date = input('Enter start date (YYYY-MM-DD): ')
    #end_date = input('Enter end date (YYYY-MM-DD): ')
    #formatted_start_date = format_date_for_output(datetime.strptime(start_date, "%Y-%m-%d"))
    #formatted_end_date = format_date_for_output(datetime.strptime(end_date, "%Y-%m-%d"))
    Month = get_non_empty_input('Enter Month of Expense')
    Year = get_non_empty_input('Enter Year of Expense: ')
    total_budgeted = calculate_total_budget(Month)
    formatted_total_budget = total_budgeted
    numeric_total_budget = float(total_budgeted.replace('₦', '').replace(',', ''))

    query_Amount = 'select Amount from Income where Month = %s AND Year = %s AND Status = %s'
    mycursor.execute(query_Amount, (Month, Year, 'Incomplete'))
    amount_result = mycursor.fetchone()

    if amount_result:
        numeric_Amount = float(amount_result[0].replace('₦', '').replace(',', ''))
        formatted_balance = numeric_Amount - numeric_total_budget
        current_datetime = format_date_for_output(datetime.now())
        #print(f'Date Range: {formatted_start_date} to {formatted_end_date}')
        print(f'Current Date is: {current_datetime}')
        print(f'Here is your insight for the Month of {Month}')
        print(f'Total Budgeted: {formatted_total_budget}')
        print(f'Your current balance is ₦{formatted_balance}')
        print(f'Your other insights go here...')

    else:
        print(f'No Income found for the specified Month, Year ans Status.')


def add_budget():
    #formatted_start_date = format_date_for_output(datetime.strptime(start_date_str, "%Y-%m-%d"))
    ID = get_non_empty_input('Enter budget ID: ')
    Date = get_non_empty_input('Enter Date of Expense')
    Month = get_non_empty_input('Enter Month of Expense: ')
    Category = get_non_empty_input('''Provide category for budget
    (Home/Rent
    Daily Living
    Transportation
    Entertainment
    Health Care
    Dues/Subscriptions
    Financial savings): ''')
    Description = get_non_empty_input('Enter description for category: ')
    Amount = get_non_empty_input('Enter Amount for category: ')
    Type = get_non_empty_input('Enter type of expense for budget(Fixed/Variable): ')
    PaymentMethod = get_non_empty_input('Enter payment method(Cash/Card/Transfer): ')
    Status = get_non_empty_input('Enter Budget status(Pending/Paid/Completed): ')

    query = 'INSERT INTO budget(ID,Date,Month,Category,Description,Amount,Type,PaymentMethod,Status) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)'
    mycursor.execute(query, (ID,Date,Month,Category,Description,Amount,Type,PaymentMethod,Status))
    mydb.commit()
    print('Budget successfully saved...')

def updateBudget():
    ID = (get_non_empty_input('Enter ID to be updated: '))
    column = get_non_empty_input('Enter the column to be updated(Date,Month,Category,Description,Amount,Type,PaymentMethod,Status): ')
    newval = get_non_empty_input('Enter the new value: ')    
    query = f'UPDATE budget SET {column} = %s WHERE ID = %s'
    mycursor.execute(query, (newval, ID))
    mydb.commit()
    print('Budget updated successfully.')   

def del_budget():
    ID = (input('Enter budget ID to be deleted: '))
    ask = input('Are you sure you want to delete this budget(Yes/No)? ')
    if ask.lower() == 'yes':
        query = 'DELETE FROM budget WHERE ID = %s'
        mycursor.execute(query, (ID,))
        mydb.commit()
        print('Budget deleted successfully...')
    else:
        print('Deletion canceled.')

def paid():
    global balance
    while True:
        mycursor.execute("SELECT ID, Month, Amount, Status, Category FROM budget WHERE Status = 'Pending' ORDER BY Month, ID")
        records = mycursor.fetchall()

        if not records:
            print('No pending expenses to pay.')
            break

        current_month = get_non_empty_input('Enter the Month for which you want to pay expenses: ')
        
        total_pending_amount = sum(int(record[2]) for record in records if record[1] == current_month)
        print(f'Total pending expenses for {current_month} is {format_with_commas(total_pending_amount)}')

        print('Pending Expenses:')
        for record in records:
            ID, Month, Amount, Category, Status = record
            if Month == current_month:
                print(f"ID: {ID}, Month: {Month}, Amount: {format_with_commas(Amount)}, Category: {Category}, Status: {Status}")

        ID = get_non_empty_input('Provide expense ID to be paid (Enter "None" to exit): ')
        
        if ID. lower() == 'None':
            break

        query_fetch_id = 'select ID, Amount from budget where ID = %s and Status = %s'
        mycursor.execute(query_fetch_id, (ID, 'Pending'))
        id_data = mycursor.fetchone()

        if id_data: 
            expense_amount = id_data[1]
            mycursor.execute("SELECT Amount FROM income WHERE Month = %s", (current_month,))
            income_balance = mycursor.fetchone()

            if income_balance is not None and expense_amount <= income_balance[0]:
                query_update_budget = 'UPDATE budget SET Status = %s WHERE ID = %s'
                mycursor.execute(query_update_budget, ('Completed', ID))
                print(f'Expense completed!')

                Amount -= expense_amount
                query_update_income = 'UPDATE income SET Amount = Amount - %s WHERE Month = %s'
                mycursor.execute(query_update_income, (expense_amount, current_month))

                mydb.commit()

                print(f'Expense ID {ID} paid successfully.')
                print(f'Your current balance for {current_month} is {format_with_commas(balance)}')

            else:
                print('Insufficient Funds. This expense is still pending...')
        else:
            print('Invalid expense ID. Please try again.')


             
def displayAll():
    mycursor.execute("SELECT ID, Date, Month, Category, Description, Amount, Type, PaymentMethod, Status FROM budget")
    records = mycursor.fetchall()
    for record in records:
        formatted_date = format_date_for_output(record[1])
        formatted_value = format_with_commas(record[5])
        formatted_record = list(record)
        formatted_record[1] = formatted_date
        formatted_record[5] = formatted_value
        print(formatted_record)

def checkBudget():
    ID = (input('Enter Budget ID: '))
    query = "SELECT ID, Date, Month, Category, Description, Amount, Type, PaymentMethod, Status FROM budget WHERE ID = %s"
    mycursor.execute(query, (ID,))
    result = mycursor.fetchone()

    if result is None:
        print('Budget not found!')
    else:
        formatted_date = format_date_for_output(result[1])
        formatted_value = format_with_commas(result[5])
        formatted_result = list(result)
        formatted_result[1] = formatted_date
        formatted_result[5] = formatted_value
        print(formatted_result)
    
        # for x in result:
        #     print(x)
   

def main():
    global balance
    operation = True
    passid = 'admin'
    password = get_non_empty_input('Enter your password: ')
    if passid == password:
        print('Welcome to Coin Save!')
        # formatted_current_datetime = format_date_for_output(datetime.now().strftime("%Y-%m-%d %H:%M:%S")) 
        # print(f'\nDate and Time: {formatted_current_datetime}')
        while operation:
            print('Operation[Balance, AddIncome, History, Insight, Budget, Update, Display, Check, Paid, Delete, Logout]')
            option = input('Select Operation: ')
            if option == 'Balance'.lower():
                main_balance() 
            elif option == 'Addincome'.lower():
                new_transfer()
            elif option == 'History'.lower():
                transaction_history()
            elif option == 'Insight'.lower():
                insight()
            elif option == 'Budget'.lower():
                add_budget()
            elif option == 'Update'.lower():
                updateBudget()
            elif option == 'Display'.lower():
                displayAll()
            elif option == 'Check'.lower():
                checkBudget()
            elif option == 'Paid'.lower():
                paid()
            elif option == 'Delete'.lower():
                del_budget()
            elif option == 'Logout'.lower():
                operation = False
                print('You are Logged out.')
                break                
    else:
        print('Incorrect Password!')

main()  

Welcome to Coin Save!
Operation[Balance, AddIncome, History, Insight, Budget, Update, Display, Check, Paid, Delete, Logout]
Budget not found!
Operation[Balance, AddIncome, History, Insight, Budget, Update, Display, Check, Paid, Delete, Logout]
Operation[Balance, AddIncome, History, Insight, Budget, Update, Display, Check, Paid, Delete, Logout]
Operation[Balance, AddIncome, History, Insight, Budget, Update, Display, Check, Paid, Delete, Logout]
Operation[Balance, AddIncome, History, Insight, Budget, Update, Display, Check, Paid, Delete, Logout]
Operation[Balance, AddIncome, History, Insight, Budget, Update, Display, Check, Paid, Delete, Logout]
Your main balance for March is ₦500,000.00
Operation[Balance, AddIncome, History, Insight, Budget, Update, Display, Check, Paid, Delete, Logout]
Operation[Balance, AddIncome, History, Insight, Budget, Update, Display, Check, Paid, Delete, Logout]
Transfer completed successfully...
Operation[Balance, AddIncome, History, Insight, Budget, Update, Di